In [2]:
!pip install chromadb -q
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00


In [18]:
#imports
import chromadb
import pandas as pd
from itertools import chain

In [6]:


client = chromadb.Client() # default in memory
collection = client.create_collection('quotes')


In [ ]:
#client.delete_collection('quotes')

In [13]:
df = pd.read_csv('sample_data/quotes.csv')
df.head(5)

,quote,person,topic
0,The only thing we have to fear is fear itself.,Franklin D. Roosevelt,Courage
1,Be yourself; everyone else is already taken.,Oscar Wilde,Individuality
2,"I think, therefore I am.",René Descartes,Philosophy
3,"To be, or not to be, that is the question.",William Shakespeare,Philosophy
4,Injustice anywhere is a threat to justice ever...,Martin Luther King Jr.,Social Justice


In [24]:
# add quotes as documents and person, topic as metadata collection itmes
# in-build embeddings
collection.add(
    ids=[str(i) for i in range(0, len(df))],
    documents=df['quote'].to_list(),
    metadatas=[{'person': p, 'topic': t} for p,t in zip(df['person'].to_list(), df['topic'].to_list())],
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 62.6MiB/s]


In [32]:
# show document and mapped embedding
collection.get(ids=['1'],include=['documents','embeddings'])


{'ids': ['1'],
 'embeddings': [[-0.007102255709469318,
   0.024232657626271248,
   0.06015212833881378,
   -0.02771695703268051,
   0.01838059164583683,
   -0.04627615213394165,
   0.08857367187738419,
   -0.06295574456453323,
   -0.033231936395168304,
   -0.0392136313021183,
   0.025631004944443703,
   -0.09541463851928711,
   -0.01652626134455204,
   -0.04381481558084488,
   0.08338762819766998,
   -0.0009665123070590198,
   0.12206076085567474,
   0.019491974264383316,
   -0.034457702189683914,
   -0.03663576394319534,
   -0.05857164412736893,
   -0.061606526374816895,
   -0.008014023303985596,
   0.0052438536658883095,
   -0.07181600481271744,
   -0.03872758150100708,
   0.02742748335003853,
   -0.05448571592569351,
   0.030683692544698715,
   -0.011821874417364597,
   0.007468836382031441,
   -0.05179515480995178,
   0.03746400400996208,
   0.05976206064224243,
   -0.08575645834207535,
   0.057704366743564606,
   -0.06409265846014023,
   -0.0634852722287178,
   0.06038544699549675

In [35]:
# query 
my_query = 'what is love'
results = collection.query(
    query_texts=my_query,
    n_results=3)


In [41]:
for doc in results['documents'][0]:
  print(doc)

Love is not only something you feel, it is something you do.
Love is an irresistible desire to be irresistibly desired.
Where there is love there is life.


In [45]:
my_query = 'what is green'
results = collection.query(
    query_texts=my_query,
    n_results=3)
for doc in results['documents'][0]:
  print(doc)

Liberty, when it begins to take root, is a plant of rapid growth.
Freedom is nothing else but a chance to be better.
Freedom is nothing but a chance to be better.


In [47]:
my_query = 'what is red'
results = collection.query(
    query_texts=my_query,
    n_results=3)
for doc in results['documents'][0]:
  print(doc)

I think, therefore I am.
Fear is a reaction. Courage is a decision.
Creativity is seeing what others see and thinking what no one else ever thought.


In [49]:
my_query = 'what to do when you are sad'
results = collection.query(
    query_texts=my_query,
    n_results=3)
for doc in results['documents'][0]:
  print(doc)

Question everything. Learn something. Answer nothing.
Do not pray for easy lives. Pray to be stronger men.
The purpose of our lives is to be happy.
